In [ ]:
import cv2
#print(cv2.__version__)

img = cv2.imread("C:\source\img\pisa1.jpg")
#print(type(img))
#print(img)

cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

#グレスケに変換
imgGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

cv2.imshow('Gray Image', imgGray)
cv2.waitKey()
cv2.destroyAllWindows()

################
## ガンマ補正 ##
################

#ガンマ値
gamma = 0.1

# 画素値の最大値
imax = imgGray.max()

# ガンマ補正
imgGamma = imax * (imgGray / imax)**(1/gamma)

cv2.imshow('Gamma Image', imgGamma)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
#写真のリサイズ

from PIL import Image

img = Image.open("img/pisa3.jpg")
def image_resize(width, image_name):
    resize_image = img.resize((width, int(width * img.size[1] / img.size[0])))  # 画像のリサイズ
    resize_image.save('img/' + image_name)  # 画像の保存

image_resize(400, 'image_resize3.jpg')

In [ ]:
import cv2
import numpy as np

# 画像を読込
src = cv2.imread('img/image_resize.jpg', cv2.IMREAD_COLOR)

# グレースケール画像へ変換
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# 2値化
retval, bw = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 輪郭を抽出
contours, hierarchy = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# 矩形検出された数（デフォルトで0を指定）
detect_count = 0

# 各輪郭に対する処理
for i in range(0, len(contours)):

    # 輪郭の領域を計算
    area = cv2.contourArea(contours[i])

    # ノイズ（小さすぎる領域）と全体の輪郭（大きすぎる領域）を除外
    if area < 1e2 or 1e5 < area:
        continue

    # 外接矩形
    if len(contours[i]) > 0:
        rect = contours[i]
        x, y, w, h = cv2.boundingRect(rect)
        cv2.rectangle(src, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # 外接矩形毎に画像を保存
        #cv2.imwrite('img/' + str(detect_count) + '.jpg', src[y:y + h, x:x + w])

        detect_count = detect_count + 1


cv2.imshow('test', src)
cv2.waitKey()
cv2.destroyAllWindows()


In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import Polygon

##### 輪郭抽出の手法B
##### 色調に差があり、それが輪郭になり得る場合
##### 2値化後に閾値で判別
def draw_contours_B(img):
    # 2値化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite( '_gray_B.jpg', gray)
    
    ret,th1 = cv2.threshold(gray,200,255,cv2.THRESH_BINARY)
    cv2.imwrite('_th1_B.jpg', gray)
    # 輪郭抽出
    contours = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = []
    #print(contours)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 10000:
            epsilon = 0.1*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            areas.append(approx)
    #cv2.drawContours(img, areas, -1, (0,255,0), 3)
    cv2.imwrite( '_boundingbox_B.jpg', img)

##### 輪郭抽出の手法C
##### 色調に明確な差がなく、線で輪郭になり得る場合
##### 2値化後に閾値で判別
def draw_contours_C(img):
    # 2値化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('_1_' + '_gray_C.jpg', gray)

    ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    cv2.imwrite('_2_' +  '_otsu_C.jpg', binary)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    binary = cv2.dilate(binary, kernel)
    cv2.imwrite('_3_' + '_dilate_C.jpg', binary)
    # 輪郭抽出
    contours = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # 一番面積が大きい輪郭を抽出
    target_contour = max(contours, key=lambda x: cv2.contourArea(x))

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img)
    ax.set_axis_off()

    for i, cnt in enumerate(contours):
        if cv2.contourArea(cnt) < 500:
            continue
        else:
            cnt = cnt.squeeze(axis=1)
            ax.add_patch(Polygon(cnt, color="b", fill=None, lw=2))
            ax.plot(cnt[:, 0], cnt[:, 1], "ro", mew=0, ms=4)
            ax.text(cnt[0][0], cnt[0][1], i, color="orange", size="20")
    plt.savefig('_4_' + '_boundingbox_C.png')
    plt.close()
    # 輪郭を構成する点を CSV に保存する。
    buf_np = target_contour.squeeze(axis=1).flatten()
    x_list = []
    y_list = []
    for i, elem in enumerate(buf_np):
        if i%2==0:
            x_list.append(elem)
        else:
            y_list.append(elem*(-1))    
    # pandasのSeries型へ一旦変換  
    x_df = pd.Series(x_list)
    y_df = pd.Series(y_list)
    # pandasのDataFrame型へ結合と共に、列名も加えて変換
    DF = pd.concat((x_df.rename(r'#X'), y_df.rename('Y')), axis=1, sort=False)
    #print(DF)
    DF.to_csv( "_target_contour_B.csv", encoding="utf-8", index=False)
    
##### 輪郭抽出の手法D
##### 色調に明確な差がなく、線で輪郭になり得る場合
##### Canny(エッジ検出アルゴリズム)で判別
def draw_contours_D(img):
    edges = cv2.Canny(img, 100, 200)
  
    cv2.imwrite('_' +'_boundingbox_D.jpg', edges)

#file_name = my_file[:-4]
img = cv2.imread('img/image_resize1.jpg')

# 輪郭描写（自作関数呼び出し）
draw_contours_B(img) # 2値化後に閾値で判別 大津アルゴリズム
#draw_contours_D(img) # Canny(エッジ検出アルゴリズム)で判別

(array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), [array([[[314, 482]],

       [[313, 483]],

       [[312, 483]],

       [[312, 484]],

       [[316, 484]]], dtype=int32), array([[[335, 480]]], dtype=int32), array([[[328, 480]],

       [[328, 481]]], dtype=int32), array([[[325, 478]],

       [[325, 479]]], dtype=int32), array([[[322, 478]],

       [[322, 482]],

       [[321, 483]],

       [[319, 483]],

       [[319, 484]],

       [[329, 484]],

       [[330, 483]],

       [[332, 483]],

       [[332, 482]],

       [[331, 481]],

       [[331, 482]],

       [[330, 483]],

       [[323, 483]],

       [[322, 482]]], dtype=int32), array([[[196, 474]],

       [[195, 475]],

       [[196, 476]],

       [[196, 478]],

       [[195, 479]],

       [[195, 480]],

       [[196, 481]],

       [[196, 483]],

     

error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\shapedescr.cpp:272: error: (-215:Assertion failed) npoints >= 0 && (depth == 5 || depth == 4) in function 'cv::contourArea'


In [21]:
import cv2
import numpy as np

# load image, change color spaces, and smoothing
img = cv2.imread('img/pisa1.jpg')
img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2XYZ)
img_HSV = cv2.GaussianBlur(img_HSV, (9, 9), 3)

# detect tulips
img_H, img_S, img_V = cv2.split(img_HSV)
_thre, img_flowers = cv2.threshold(img_H, 140, 255, cv2.THRESH_BINARY)
cv2.imwrite('tulips_mask.jpg', img_flowers)

# find tulips
labels, contours, hierarchy = cv2.findContours(img_flowers, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

for i in range(0, len(contours)):
    if len(contours[i]) > 0:

        # remove small objects
        if cv2.contourArea(contours[i]) < 500:
            continue

        cv2.polylines(img, contours[i], True, (255, 255, 255), 5)

# save
cv2.imwrite('tulips_boundingbox.jpg', img_HSV)

True

In [237]:
import cv2
import numpy as np

# 白色の検出
def detect_white_color(img):
    # HSV色空間に変換
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 白色のHSVの値域
    hsv_min = np.array([0,0,100])
    hsv_max = np.array([180,45,255])
    mask = cv2.inRange(hsv, hsv_min, hsv_max)

    # マスキング処理
    masked_img = cv2.bitwise_and(img, img, mask=mask)

    return mask, masked_img

# 入力画像の読み込み
img = cv2.imread('img/pisa1.jpg')

# 白色検出
white_mask, white_masked_img = detect_white_color(img)

# 結果を出力
cv2.imwrite('img/pisa_white_mask.jpg', white_mask)
cv2.imwrite('img/pisa_white_mask_img.jpg', white_masked_img)

#ガウシアンフィルターでぼかす
blurred_img = cv2.GaussianBlur(white_masked_img,(5,5),10)

imgray = cv2.cvtColor(white_masked_img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,115,255,0)
im, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

max = cv2.contourArea(contours[0])

for cnt in contours:
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, (0,255,0), 2)

cv2.imwrite('img/pisa_rect.jpg', img)

True

In [309]:
##色々フィルタリングしたけど結局使われないかわいそうなスクリプト

import cv2
import numpy as np

# 白色の検出
def detect_white_color(img):
    # HSV色空間に変換
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 白色のHSVの値域
    hsv_min = np.array([0,0,100])
    hsv_max = np.array([180,45,255])
    mask = cv2.inRange(hsv, hsv_min, hsv_max)

    # マスキング処理
    masked_img = cv2.bitwise_and(img, img, mask=mask)

    return mask, masked_img

#面積が最大の長方形のインデックスを返す関数
def detect_max_rect(img,contours):
    #最大面積をとるcontoursのインデックス
    max_idx = 0
    
    for i in range(len(contours)):
        if cv2.contourArea(contours[max_idx]) < cv2.contourArea(contours[i]):
            max_idx = i
    
    return max_idx

# 入力画像の読み込み
img = cv2.imread('img/pisa2.jpg')

# 白色検出
white_mask, white_masked_img = detect_white_color(img)

# 結果を出力
cv2.imwrite('img/pisa_white_mask.jpg', white_mask)
cv2.imwrite('img/pisa_white_mask_img.jpg', white_masked_img)

#ガウシアンフィルターでぼかす
blurred_img = cv2.GaussianBlur(white_masked_img,(5,5),10)

imgray = cv2.cvtColor(blurred_img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,120,255,0)
kernel = np.ones((9,9),np.uint8)
im, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

erosion = cv2.erode(thresh,kernel,iterations = 1)
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

target_contour = contours[detect_max_rect(imgray,contours)]

rect = cv2.minAreaRect(target_contour)
box = cv2.boxPoints(rect)
box = np.int0(box)
#緑の四角で囲む
#cv2.drawContours(img, [box], 0, (0,255,0), 2)
#白で塗りつぶす
cv2.drawContours(img, [box], 0, (255,255,255), -1)

print(rect)

cv2.imwrite('img/pisa_rect.jpg', img)

((213.31179809570312, 324.1302490234375), (473.1288757324219, 195.5601806640625), -82.20226287841797)


True

In [85]:
import cv2
import numpy as np
from PIL import Image

# 白色の検出
def detect_white_color(img):
    # HSV色空間に変換
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 白色のHSVの値域
    hsv_min = np.array([0,0,100])
    hsv_max = np.array([180,45,255])
    mask = cv2.inRange(hsv, hsv_min, hsv_max)

    # マスキング処理
    masked_img = cv2.bitwise_and(img, img, mask=mask)

    return mask, masked_img

#面積が最大の長方形のインデックスを返す関数
def detect_max_rect(img,contours):
    #最大面積をとるcontoursのインデックス
    max_idx = 0
    
    for i in range(len(contours)):
        if cv2.contourArea(contours[max_idx]) < cv2.contourArea(contours[i]):
            max_idx = i
    
    return max_idx

#切り出し関数
def crop_rect(img, rect):
    center, size, angle = rect
    center = tuple(map(int, center))  # float -> int
    size = tuple(map(int, size))  # float -> int
    h, w = img.shape[:2]  # 画像の高さ、幅
    
    #cv2.circle(img,center,5,(0,0,255))

    #rectの高さと幅を取得
    height,width=rect[1]
    height,width=int(height),int(width)
    height+=90
        
    #angleが水平方向の線とのなす角を示すので，＋９０している
    if angle<0:
        angle+=90
    
    # affine行列を用いて画像を回転させる
    M = cv2.getRotationMatrix2D(center, angle, 1)
    rotated = cv2.warpAffine(img, M, (w, h))

    # 切り抜く
    cropped = cv2.getRectSubPix(rotated, (width,height), center)

    return cropped

#回転させる関数
def rotate_img(img,rect):
    #高さ，幅から画面の中心を計算
    _height = img.shape[0]                         
    _width = img.shape[1]                       
    _center = (int(_width/2), int(_height/2))
    
    _angle=rect[2]
    #angleが水平方向の線とのなす角を示すので，＋９０している
    if _angle<0:
        _angle+=90
    
    # affine行列を用いて画像を回転させる
    _M = cv2.getRotationMatrix2D(_center, _angle, 1)
    _rotated_img = cv2.warpAffine(img, _M, (_width, _height))
    
    return _rotated_img

#画像補完
def inpaint_img(img):
    
    _mask = img

    #白で塗りつぶす
    cv2.drawContours(_mask, [box], 0, (255,255,255), -1)
    _maskGray = cv2.cvtColor(_mask,cv2.COLOR_BGR2GRAY)
    ret,_thresh = cv2.threshold(_maskGray,254,255,0)
    
    #画像補正
    dst = cv2.inpaint(img, _thresh, 3, cv2.INPAINT_NS)
    
    return dst
        
#トリミングしたピサを補完した背景画像に貼り付ける
def paste_pisa(rect):
    #x,yにトリムしたピサのcenter座標が入る
    x,y=rect[0]
    
    #左上に持っていきたいので，半分の幅wと高さhをx,yから引く
    _w,_h=rect[1]
    #print(h,w)
    x -= _h/2
    y -= _w/2
    
    x,y=int(x),int(y)
    
    source_img = Image.open('img/pisa_rect_crop.jpg')
    canvas_img = Image.open('img/pisa_rect_inpaint.jpg')
    
    #inpaintした画像にトリムしたピサをペースト
    canvas_img.paste(source_img, (x,y))
    #画像を保存
    canvas_img.save('img/vertical_pisa.jpg')

# 入力画像の読み込み
img = cv2.imread('img/pisa1.jpg')

# 白色検出
white_mask, white_masked_img = detect_white_color(img)

# 結果を出力
cv2.imwrite('img/pisa_white_mask.jpg', white_mask)
cv2.imwrite('img/pisa_white_mask_img.jpg', white_masked_img)

#ガウシアンフィルターでぼかす
blurred_img = cv2.GaussianBlur(white_masked_img,(5,5),10)

imgray = cv2.cvtColor(blurred_img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,115,255,0)
im, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

target_contour = contours[detect_max_rect(img,contours)]

rect = cv2.minAreaRect(target_contour)
box = cv2.boxPoints(rect)
box = np.int0(box)
#緑の四角で囲む
#cv2.drawContours(img, [box], 0, (0,255,0), 2)

croped_img = crop_rect(img,rect)
#rotated_img = rotate_img(img,rect)

cv2.imwrite('img/pisa_rect.jpg', img)
cv2.imwrite('img/pisa_rect_crop.jpg', croped_img)
#cv2.imwrite('img/pisa_rect_rotate.jpg', rotated_img)

inpaint_img =  inpaint_img(img)
cv2.imwrite('img/pisa_rect_inpaint.jpg', inpaint_img)
paste_pisa(rect)

In [91]:
import sys

print(sys.version)

3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]
